In [3]:
from typing import Any, Type

from langchain.chat_models import ChatOpenAI
from langchain.agents import initialize_agent, AgentType
from langchain.tools import BaseTool
from pydantic import BaseModel, Field

# 3.5는 결과가 틀림
llm = ChatOpenAI(temperature=0.1)

# pydantic의 type
class CalculatorToolArgsSchema(BaseModel):
    a: float = Field(description="The first number")
    b: float = Field(description="The second number")


# OpenAI에서 사용할 Tool
class CalculatorTool(BaseTool):
    name = "CalculatorTool"
    description = """
    Use this to perform sums of two numbers.
    The first and second arguments should be numbers.
    Only receives two arguments.
    """
    # Schema를 정의 해야 함. pydantic
    args_schema: Type[CalculatorToolArgsSchema] = CalculatorToolArgsSchema

    def _run(self, a, b):
        return a + b


"""
agent에 input을 줄 때 runtime은 LLM으로 부터 get_action을 호출해서 LLM이 선택한 Tool을 돌림
AgentFinish가 될떄까지 loop를 돌리기 떄문에 Cost가 많이 나올 수 있음.
가끔 LLM이 Json이 아닌 Text로 답할 때가 있음. 그러땐 Agent가 에러가 남
"""
# llm, tool, agent를 전달해야 함.
agent = initialize_agent(
    llm=llm,
    verbose=True,
    agent=AgentType.OPENAI_FUNCTIONS, # OpenAI Agent
    handle_parsing_errors=True, # LLM의 output 때문에 Parsing Error가 발생하면 이를 수정하려고 함.
    tools=[
        CalculatorTool(),
    ],
)


# LLM은 산수를 직접 할 수 없음
prompt = "Cost of $355.39 + $924.87 + $721.2 + $1940.29 + $573.63 + $65.72 + $35.00 + $552.00 + $76.16 + $29.12" # $5,273.38

# LangChain이 tool을 포함한 prompt를 만들어서 LLM에 전달함.
agent.invoke(prompt) # $4923.38



> Entering new AgentExecutor chain...

Invoking: `CalculatorTool` with `{'a': 355.39, 'b': 924.87}`


1280.26
Invoking: `CalculatorTool` with `{'a': 1280.26, 'b': 721.2}`


2001.46
Invoking: `CalculatorTool` with `{'a': 2001.46, 'b': 1940.29}`


3941.75
Invoking: `CalculatorTool` with `{'a': 3941.75, 'b': 573.63}`


4515.38
Invoking: `CalculatorTool` with `{'a': 4515.38, 'b': 65.72}`


4581.1
Invoking: `CalculatorTool` with `{'a': 4581.1, 'b': 35.0}`


4616.1
Invoking: `CalculatorTool` with `{'a': 4616.1, 'b': 552.0}`


5168.1
Invoking: `CalculatorTool` with `{'a': 5168.1, 'b': 76.16}`


5244.26
Invoking: `CalculatorTool` with `{'a': 5244.26, 'b': 29.12}`


5273.38The total cost of $355.39 + $924.87 + $721.2 + $1940.29 + $573.63 + $65.72 + $35.00 + $552.00 + $76.16 + $29.12 is $5273.38.

> Finished chain.


{'input': 'Cost of $355.39 + $924.87 + $721.2 + $1940.29 + $573.63 + $65.72 + $35.00 + $552.00 + $76.16 + $29.12',
 'output': 'The total cost of $355.39 + $924.87 + $721.2 + $1940.29 + $573.63 + $65.72 + $35.00 + $552.00 + $76.16 + $29.12 is $5273.38.'}